In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

In [21]:
data_path = os.path.join('..','data','NHANES2','Vertebrae')

mode = 'test'

if mode == 'train':
    with open(os.path.join(data_path,'..','data_split','train.txt'),'r') as f:
        id_list = [line[0:6] for line in f.readlines()]
elif mode == 'val':
    with open(os.path.join(data_path,'..','data_split','val.txt'),'r') as f:
        id_list = [line[0:6] for line in f.readlines()]
elif mode == 'test':
    with open(os.path.join(data_path,'..','data_split','test.txt'),'r') as f:
        id_list = [line[0:6] for line in f.readlines()]

gt_path = os.path.join(data_path,'gts')
gt_filenames = [filename for filename in os.listdir(gt_path) if filename.split('_')[0] in id_list]

In [22]:
os.makedirs(os.path.join(data_path,'resnet_background_sample_areas'),exist_ok=True)

for id in id_list[:]:

    whole_gt = []

    for filename in [f for f in gt_filenames if f.split('_')[0] == id]:
        gt = np.load(os.path.join(data_path,'gts',filename)).astype(np.float32)
        if type(whole_gt) == list:
            whole_gt = gt
        else:
            whole_gt += gt

    filtered_gt = gaussian_filter(whole_gt,sigma=40)
    rough_border = (filtered_gt > 0.1).astype(np.float32)

    filtered_gt_2 = gaussian_filter(rough_border,sigma=50)
    rough_border_2 = (filtered_gt_2 > 0.05).astype(np.uint8)

    sample_area = rough_border_2-rough_border

    np.save(os.path.join(data_path,'resnet_background_sample_areas',id+'.npy'),sample_area)